In [ ]:
!pip install d2l

In [ ]:
import math
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size

    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    print(W_xh.shape)
    W_hh = normal((num_hiddens, num_hiddens))
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device),)

def rnn(inputs, state, params):
    # `inputs`的形状：(`时间步数量`, `批量大小`, `词表大小`)
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    loop_nums = 0
    # `X`的形状：(`批量大小`, `词表大小`)
    for X in inputs:  # 32 * 28
        loop_nums += 1
        H = torch.tanh(torch.mm(X, W_xh) + torch.mm(H, W_hh) + b_h)
        Y = torch.mm(H, W_hq) + b_q 
        outputs.append(Y)
    return torch.cat(outputs, dim=0), (H,)


class RNNModelScratch:
    def __init__(self, vocab_size, num_hiddens, device, get_params,
                 init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):  # 因为这个模型没有继承自nn.Module模块，因此只能通过实现__call__函数来做forward
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)  # 得到输入的one-hot encoding
        return self.forward_fn(X, state, self.params)  # 回调函数

    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

# 定义model
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)

# LSTM提出的clipping：梯度裁剪技术
def grad_clipping(net, theta):
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = 0.0
    results = 0.0
    for p in params:
      results += torch.sum(p.grad**2)
    norm = torch.sqrt(results)
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm


def predict_ch8(prefix, num_preds, net, vocab, device):
    """在`prefix`后面生成新字符。"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape(
        (1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测`num_preds`步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损失之和, 标记数量
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化`state`
            state = net.begin_state(batch_size=X.shape[0], device=device)   # 初始化其实隐藏状态H0
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # `state`对于`nn.GRU`是个张量
                state.detach_()
            else:
                # `state`对于`nn.LSTM`或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)  # __call__
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了`mean`函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()


def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.Adam(net.parameters(), lr)  
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(net, train_iter, loss, updater, device,
                                     use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller'))
            animator.add(epoch + 1, [ppl])
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 标记/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('traveller'))

num_epochs, lr = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

In [ ]:
import collections
import re
from d2l import torch as d2l

d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine(): 
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    print(lines)
    results = []
    for line in lines:
      results.append(re.sub('[^A-Za-z]+', ' ', line).strip().lower())
    return results

lines = read_time_machine()
# print(f'# text lines: {len(lines)}')
# print(lines[0])
def tokenize(lines, token='word'):
    results = []
    if token == 'word':
        for line in lines:
          arr = line.split()
          results.append(arr)
        return results
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知令牌类型：' + token)

tokens = tokenize(lines)  # 二维list，list[i]都表示原来文本的一行
# tokens = tokenize(lines, 'char')  # 测试结果
# print(tokens)
# for i in range(11):
#     print(tokens[i])

class Vocab:  # 词汇表的目的就是生成一个hash table，每一个token都应该对应一个唯一的索引值 [token, index]
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        
        counter = count_corpus(tokens)
        # print(type(counter))
        # for key, value in counter.items():
        #   print(key, value)   # 打印counter.items()字典中的每个元素
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)   # 默认是升序，reverse=True为降序
        # print(type(self.token_freqs))  # 返回一个list对象
        # print(self.token_freqs)

        self.unk = 0
        self.idx_to_token = ['unk']
        self.token_to_idx =  dict()  # idx_to_token 为list类型 token_to_idx为dict类型
        self.token_to_idx['unk'] = 0
        for token, _ in self.token_freqs:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.token_to_idx)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple, dict)):
            return self.token_to_idx.get(tokens, self.unk)   # 访问token_to_idx字典，如果key在字典中，则返回其对应的value，如果key不在，则返回self.unk标记
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple,dict)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(tokens):
    # 这里的 `tokens` 是 1D 列表或 2D 列表
    if len(tokens) == 0 or isinstance(tokens[0], list):   # isinstance 就是判断tokens[0]是不是list类型
        # 将标记列表展平成使用标记填充的一个列表
        results = []
        for line in tokens:
          for t in line:
            results.append(t)
        tokens = results
    return collections.Counter(tokens)

vocab =Vocab(tokens)
# for i in [0, 10]:
#     print('words:', tokens[i])
#     print('indices:', vocab[tokens[i]])  # 这里默认调用__getitem__方法

def load_corpus_time_machine(max_tokens=-1):
    """返回时光机器数据集的标记索引列表和词汇表。"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')  # 二维数组
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每一个文本行，不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    # corpus = [vocab[token] for line in tokens for token in line]  # 把整个文本表示为一个一维index list
    corpus = []
    for line in tokens:
      for token in line:
        corpus.append(vocab[token])
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
print(corpus)
len(corpus), len(vocab)

In [ ]:
import random
import torch
from d2l import torch as d2l

tokens = d2l.tokenize(d2l.read_time_machine())
# 因为每个文本行不一定是一个句子或一个段落，因此我们把所有文本行连接到一起
corpus = [token for line in tokens for token in line]
vocab = d2l.Vocab(corpus)
vocab.token_freqs[:10]

len(corpus), len(corpus[:-1]), len(corpus[1:])

bigram_tokens = []
for pair in zip(corpus[:-1], corpus[1:]):
  bigram_tokens.append(pair)
bigram_vocab = d2l.Vocab(bigram_tokens)
bigram_vocab.token_freqs[:10]


# def seq_data_iter_sequential(corpus, batch_size, num_steps):
#     """使用顺序分区生成一个小批量子序列。"""
#     # 从随机偏移量开始划分序列
#     offset =  1 # random.randint(0, num_steps)
#     print(offset)
#     num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
#     print(num_tokens)
#     Xs = torch.tensor(corpus[offset:offset + num_tokens])
#     Ys = torch.tensor(corpus[offset + 1:offset + 1 + num_tokens])
#     Xs, Ys = Xs.reshape(batch_size, -1), Ys.reshape(batch_size, -1)
#     num_batches = Xs.shape[1] // num_steps
#     print(num_batches)
#     for i in range(0, num_steps * num_batches, num_steps):
#         print(i)
#         X = Xs[:, i:i + num_steps]
#         Y = Ys[:, i:i + num_steps]
#         yield X, Y


def seq_data_iter_random(corpus, batch_size, num_steps):
    """使用随机抽样生成一个小批量子序列。"""
    # 从随机偏移量（包括`num_steps - 1`）开始对序列进行分区
    corpus = corpus[random.randint(0, num_steps - 1):]
    # 减去1，因为我们需要考虑标签
    num_subseqs = (len(corpus) - 1) // num_steps
    # 长度为`num_steps`的子序列的起始索引
    initial_indices = list(range(0, num_subseqs * num_steps, num_steps))
    # 在随机抽样中，
    # 迭代过程中两个相邻的、随机的小批量中的子序列不一定在原始序列上相邻
    random.shuffle(initial_indices)

    def data(pos):
        # 返回从`pos`开始的长度为`num_steps`的序列
        return corpus[pos:pos + num_steps]

    num_batches = num_subseqs // batch_size
    for i in range(0, batch_size * num_batches, batch_size):
        # 这里，`initial_indices`包含子序列的随机起始索引
        initial_indices_per_batch = initial_indices[i:i + batch_size]
        X = [data(j) for j in initial_indices_per_batch]
        Y = [data(j + 1) for j in initial_indices_per_batch]
        yield torch.tensor(X), torch.tensor(Y)

my_seq = list(range(35))
print(my_seq)
for X, Y in seq_data_iter_random(my_seq, batch_size=2, num_steps=5):
    print('X: ', X, '\nY:', Y)

In [ ]:
import torch
import torch.nn.functional as F
a = torch.tensor([[27, 28, 29, 30, 31],
        [ 2,  3,  4,  5,  6]]) 
print(a.T.shape)
F.one_hot(a)
F.one_hot(a.T)